In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import random
from operator import add

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [7]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param K {I} >=0;   # capacity of machine 

var X {I,J} >=0;   # Decision var 1
var MinBnft;       # Decision var 2

maximize Benefit: MinBnft;

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= K[i];
s.t. ProdTime {        j in J}: sum {i in I}        X[i,j] <= 1;
s.t. MacBnft  {i in I        }: sum {j in J} B[j] * X[i,j] >= MinBnft;
s.t. Arrange  {i in I, j in J}:                     X[i,j] >= 0;

s.t. MinBenefit : MinBnft >=0;
''')

## Step 5: Define the initial data

In [8]:
# batch 要做成:
# [ num_machine, num_job, cost, benefit, capacity ]

it_n = 100

def batch (num_machine, num_job, relation_bc, capacity):
    
    m = num_machine
    n = num_job
    
    list_c1 = []    # 內含 100組 n個 job的 cost 組合
    list_b1 = []    # 內含 100組 n個 job的 bnft 組合
    list_k1 = []    # 內含 100組 由 n個 job 加總算出的 k
    
    for i in range(it_n):
        
        list_c2 = []   # 內含 n個 job的 cost
        list_b2 = []   # 內含 n個 job的 bnft
        
        for j in range(n):
            c = int(random.randint(0,50))
            list_c2.append(c)

            if   relation_bc == 'L':
                b = c

            elif relation_bc == 'X':
                b = c**2

            elif relation_bc == 'A':
                b = c**(1/2)

            elif relation_bc == 'R':
                b = random.randint(0,50)  
            list_b2.append(b)

        if   capacity =='N':
            k = 10000000 # no capacity

        elif capacity =='L':
            k = sum(list_c2) / m

        elif capacity =='T':
            k = sum(list_c2) * (0.75) / m
    
        list_c1.append(list_c2)
        list_b1.append(list_b2)
        list_k1.append(k)
    
    return  m, n, list_c1, list_b1, list_k1

In [39]:
# Test OK! 不要在正式 run 100組時  print，會很慢 >"<

instance = batch( 5, 20, 'A', 'T' )
print(instance[2])
print(instance[3])
print(instance[4])

[[5, 6, 29, 39, 48, 9, 7, 21, 23, 4, 38, 33, 29, 1, 45, 35, 42, 20, 43, 33]]
[[2.23606797749979, 2.449489742783178, 5.385164807134504, 6.244997998398398, 6.928203230275509, 3.0, 2.6457513110645907, 4.58257569495584, 4.795831523312719, 2.0, 6.164414002968976, 5.744562646538029, 5.385164807134504, 1.0, 6.708203932499369, 5.916079783099616, 6.48074069840786, 4.47213595499958, 6.557438524302, 5.744562646538029]]
[76.5]


In [10]:
machine_job  = [(5,50),(15,50),(15,500),(50,500)] # ,(5,500)
capacity     = ['L','T','N']
relation_bc  = ['L','A','X','R']

## Step 6: Model Build-up & Solution

In [11]:
# 迭代 instance

MinBenefit_ijk_sn_1 = []
MinBenefit_ijk_sn_2 = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]
    
    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for k in capacity:    
        for j in relation_bc:

            instance = batch( i[0], i[1], j, k )
            
            MinBenefit_1 = []
            MinBenefit_2 = []
            
            for p in range(it_n):      
                
                # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均
                
                # amplpy DataFrame
                ampl.setData(DataFrame(
                index=[('J', job)], 
                columns=[
                    ('B', instance[3][p]), 
                    ('C', instance[2][p])
                ]
                ))

                # Pandas DataFrame
                df = pd.DataFrame({
                    'K': [ instance[4][p] ]*num_machine
                }, 
                    index = machine
                )
                ampl.setData(DataFrame.fromPandas(df))

                # Solve the problem
                ampl.solve()

                '''
                # Create a DataFrame with Decision variables
                Var = ampl.getVariable('X').getValues().toPandas()
                print(Var)
                '''

                # Display the objective value
                MinBnft_1 = ampl.getObjective('Benefit')
                MinBnft_1 = MinBnft_1.value()
                MinBenefit_1 = np.append(MinBenefit_1, MinBnft_1) # numpy.ndarray
                
                
                # ====== Model 2 ======= : 一組一組用 CHBF算，每種senario要算 100組 再平均
                
                # 將 job 按照 cost (workload) 排序

                list_c = instance[2][p]
                list_b = instance[3][p]

                list_cb = list(zip(list_c, list_b))        # [3,5]  [8,1]  -> [(3,8),(5,1)]
                s_cb    = sorted(list_cb, reverse = True)  # [(3,8),(5,1)] -> [(5,1),(3,8)]
                o_cb    = list(zip(*s_cb))                 # [(5,1),(3,8)] -> [(5,3),(1,8)]

                list_c  = list(o_cb[0])                    # [5,3]
                list_b  = list(o_cb[1])                    # [1,8]

                mach = []                 # machine
                ca = instance[4][p]

                for r in range(num_machine):

                    mach.append([])    # machine set中安排一台台 machine 被 append 進來
                    mach[r].append(0)  # 初始的 benefit和 是 0
                    mach[r].append(ca) # 初始的 capacity  是 ca

                for s in range(num_job):

                    mach = sorted(mach)                           # 按照 benefit和 由小而大排序
                    # print(machine)

                    for r in range(num_machine):                  # 由最小 benefit和 的先開始

                        if list_c[s] <= mach[r][1]:               # machine r 還有 capacity 餘裕

                            # print(mach[r])
                            mach[r][1] -= list_c[s]
                            mach[r][0] += list_b[s]
                            # print(mach[r])
                            # print('=================')
                            break
                
                mach = sorted(mach)                               # 再一次 按照 benefit和 由小而大排序
                MinBnft_2 = mach[0][0]                            # 將最小的 benefit和的 machine 取出來
                MinBenefit_2 = np.append(MinBenefit_2, MinBnft_2) # numpy.ndarray
                
                
            # ====== Model 1 Objective value: ====== #
            
            list_M1          = list(MinBenefit_1)
            MinBenefit_100_1 = sum(list_M1) / len(list_M1)
            MinBenefit_ijk_sn_1.append(MinBenefit_100_1)   # i*j*k 種 senario 迭代
            
            # ====== Model 2 Objective value: ====== #
            
            list_M2         = list(MinBenefit_2)
            MinBenefit_100_2 = sum(list_M2) / len(list_M2)
            MinBenefit_ijk_sn_2.append(MinBenefit_100_2)   # i*j*k 種 senario 迭代
            
            print('=========================' + str(i) + str(j) + str(k) + '=========================') # 分隔           
            
print(MinBenefit_ijk_sn_1) # list
print(MinBenefit_ijk_sn_2) # list

MinBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, MinBenefit_ijk_sn_2, MinBenefit_ijk_sn_1))
MinBenefit_ijk_sn_ratio

CPLEX 12.8.0.0: optimal solution; objective 254
85 dual simplex iterations (65 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 220
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 236.2
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 254.2
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 246
7 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 273
8 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 251.6
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 240.6
4 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 241.2
5 dual simplex iterations (2 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 225.2
4 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 252.2
4 dual simplex iterations (0 in phase I)
CPLEX

CPLEX 12.8.0.0: optimal solution; objective 46.14053932
17 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 48.75600464
3 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44.06557219
18 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.84087449
14 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 47.29867747
18 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.75176915
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 52.38521132
20 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44.08066817
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 45.48630392
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44.41537916
19 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optim

CPLEX 12.8.0.0: optimal solution; objective 181.2
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 167.4
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 179.25
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 201.6
9 dual simplex iterations (3 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 197.7
11 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 185.4
7 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 176.4
14 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 187.2
11 dual simplex iterations (3 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 192.75
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 174.45
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 183.75
8 dual simplex iterations (0 

CPLEX 12.8.0.0: optimal solution; objective 225.7670455
38 dual simplex iterations (28 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 227.5705882
46 dual simplex iterations (34 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 220.7026316
62 dual simplex iterations (40 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 205.5673913
56 dual simplex iterations (31 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 213.129
44 dual simplex iterations (28 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 234.6268293
47 dual simplex iterations (30 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 256.1560976
49 dual simplex iterations (16 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 239.4655172
38 dual simplex iterations (24 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 242.7590909
35 dual simplex iterations (22 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 244.6361111
40 dual simplex iterations (27 in phase I)
CPLEX 12.8.0.0

CPLEX 12.8.0.0: optimal solution; objective 44.21674251
2 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 49.62630062
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44.68014777
2 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 47.60222342
3 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 45.11108485
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44.70344285
5 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 51.55844908
4 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 47.86915585
2 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 52.7723414
4 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 44.69731036
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solutio

...

CPLEX 12.8.0.0: optimal solution; objective 245.08
338 dual simplex iterations (89 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 248.88
152 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 244.88
138 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 232.88
142 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 257.36
159 dual simplex iterations (10 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 252.16
166 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 244.04
141 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 246
137 dual simplex iterations (11 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 237.18
166 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 247.5
130 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 250.92
161 dual s

CPLEX 12.8.0.0: optimal solution; objective 246
159 dual simplex iterations (10 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 255.06
157 dual simplex iterations (6 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 245.32
158 dual simplex iterations (7 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 254.38
158 dual simplex iterations (8 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 260.7
151 dual simplex iterations (8 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 247.26
132 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 245.94
154 dual simplex iterations (6 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 248.78
148 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 238.36
158 dual simplex iterations (6 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 247.3
156 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 252.84
148 dual simp

CPLEX 12.8.0.0: optimal solution; objective 46.98593794
123 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.62219826
135 dual simplex iterations (3 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 45.55463468
143 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.28341586
80 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.5124602
134 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.40038772
164 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 46.59460114
132 dual simplex iterations (0 in phase I)
=========================(50, 500)AN=========================
CPLEX 12.8.0.0: optimal solution; objective 7878.28
0 simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 8286.2
1019 dual simplex iterations (627 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 8086.18
974 dual s

CPLEX 12.8.0.0: optimal solution; objective 247.1
174 dual simplex iterations (3 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 248.42
170 dual simplex iterations (11 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 252.38
140 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 247.66
144 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 238.42
147 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 242.36
138 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 251.9
133 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 243.58
178 dual simplex iterations (8 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 246.98
152 dual simplex iterations (5 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 239.92
129 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 248.54
128 dual s

[0.98904026786921462,
 0.9710515790434352,
 0.99554439981834519,
 0.62909955870928425,
 0.98946028458310376,
 0.81224979742769765,
 0.96979216675517099,
 0.45375606110931488,
 0.9948461845785056,
 0.98676892593006627,
 0.99868686681302421,
 0.94840813195243578,
 0.8996297328502606,
 0.84774601322096232,
 0.90453898105054886,
 0.50823868181091025,
 0.93749749685605177,
 0.82093522218658455,
 0.78646038570501497,
 0.36263615743834499,
 0.96496691634445853,
 0.95301246857175892,
 0.98103904008245701,
 0.81218152100759589,
 0.99913208332000192,
 0.99499157422921047,
 0.99985854721212797,
 0.75760916983465754,
 0.999131927761014,
 0.8099017754716763,
 0.99387530435743809,
 0.51764282515020066,
 0.99930705247938534,
 0.99591491801474841,
 0.99995974541665167,
 0.98402409084862141,
 0.99217328168829388,
 0.97424293991947175,
 0.99791475168036536,
 0.6474375993627981,
 0.9913237186073488,
 0.8021312045262432,
 0.97557310531899477,
 0.4884543974512775,
 0.99491672395412634,
 0.97688109445951088

In [12]:
MinBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, MinBenefit_ijk_sn_2, MinBenefit_ijk_sn_1))
MinBenefit_ijk_sn_ratio

[0.98904026786921462,
 0.9710515790434352,
 0.99554439981834519,
 0.62909955870928425,
 0.98946028458310376,
 0.81224979742769765,
 0.96979216675517099,
 0.45375606110931488,
 0.9948461845785056,
 0.98676892593006627,
 0.99868686681302421,
 0.94840813195243578,
 0.8996297328502606,
 0.84774601322096232,
 0.90453898105054886,
 0.50823868181091025,
 0.93749749685605177,
 0.82093522218658455,
 0.78646038570501497,
 0.36263615743834499,
 0.96496691634445853,
 0.95301246857175892,
 0.98103904008245701,
 0.81218152100759589,
 0.99913208332000192,
 0.99499157422921047,
 0.99985854721212797,
 0.75760916983465754,
 0.999131927761014,
 0.8099017754716763,
 0.99387530435743809,
 0.51764282515020066,
 0.99930705247938534,
 0.99591491801474841,
 0.99995974541665167,
 0.98402409084862141,
 0.99217328168829388,
 0.97424293991947175,
 0.99791475168036536,
 0.6474375993627981,
 0.9913237186073488,
 0.8021312045262432,
 0.97557310531899477,
 0.4884543974512775,
 0.99491672395412634,
 0.97688109445951088

In [15]:
machine_job  = [(5,500)]

In [16]:
# 迭代 instance

MinBenefit_ijk_sn_1 = []
MinBenefit_ijk_sn_2 = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]
    
    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for j in relation_bc:
        for k in capacity:

            instance = batch( i[0], i[1], j, k )
            
            MinBenefit_1 = []
            MinBenefit_2 = []
            
            for p in range(it_n):      
                
                # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均
                
                # amplpy DataFrame
                ampl.setData(DataFrame(
                index=[('J', job)], 
                columns=[
                    ('B', instance[3][p]), 
                    ('C', instance[2][p])
                ]
                ))

                # Pandas DataFrame
                df = pd.DataFrame({
                    'K': [ instance[4][p] ]*num_machine
                }, 
                    index = machine
                )
                ampl.setData(DataFrame.fromPandas(df))

                # Solve the problem
                ampl.solve()

                '''
                # Create a DataFrame with Decision variables
                Var = ampl.getVariable('X').getValues().toPandas()
                print(Var)
                '''

                # Display the objective value
                MinBnft_1 = ampl.getObjective('Benefit')
                MinBnft_1 = MinBnft_1.value()
                MinBenefit_1 = np.append(MinBenefit_1, MinBnft_1) # numpy.ndarray
                
                
                # ====== Model 2 ======= : 一組一組用 CHBF算，每種senario要算 100組 再平均
                
                # 將 job 按照 cost (workload) 排序

                list_c = instance[2][p]
                list_b = instance[3][p]

                list_cb = list(zip(list_c, list_b))        # [3,5]  [8,1]  -> [(3,8),(5,1)]
                s_cb    = sorted(list_cb, reverse = True)  # [(3,8),(5,1)] -> [(5,1),(3,8)]
                o_cb    = list(zip(*s_cb))                 # [(5,1),(3,8)] -> [(5,3),(1,8)]

                list_c  = list(o_cb[0])                    # [5,3]
                list_b  = list(o_cb[1])                    # [1,8]

                mach = []                 # machine
                ca = instance[4][p]

                for r in range(num_machine):

                    mach.append([])    # machine set中安排一台台 machine 被 append 進來
                    mach[r].append(0)  # 初始的 benefit和 是 0
                    mach[r].append(ca) # 初始的 capacity  是 ca

                for s in range(num_job):

                    mach = sorted(mach)                           # 按照 benefit和 由小而大排序
                    # print(machine)

                    for r in range(num_machine):                  # 由最小 benefit和 的先開始

                        if list_c[s] <= mach[r][1]:               # machine r 還有 capacity 餘裕

                            # print(mach[r])
                            mach[r][1] -= list_c[s]
                            mach[r][0] += list_b[s]
                            # print(mach[r])
                            # print('=================')
                            break

                mach = sorted(mach)                               # 再一次 按照 benefit和 由小而大排序
                MinBnft_2 = mach[0][0]                            # 將最小的 benefit和的 machine 取出來
                MinBenefit_2 = np.append(MinBenefit_2, MinBnft_2) # numpy.ndarray
                
                
            # ====== Model 1 Objective value: ====== #
            
            list_M1          = list(MinBenefit_1)
            MinBenefit_100_1 = sum(list_M1) / len(list_M1)
            MinBenefit_ijk_sn_1.append(MinBenefit_100_1)   # i*j*k 種 senario 迭代
            
            # ====== Model 2 Objective value: ====== #
            
            list_M2         = list(MinBenefit_2)
            MinBenefit_100_2 = sum(list_M2) / len(list_M2)
            MinBenefit_ijk_sn_2.append(MinBenefit_100_2)   # i*j*k 種 senario 迭代
            
            print('=========================' + str(i) + str(j) + str(k) + '=========================') # 分隔           
            
print(MinBenefit_ijk_sn_1) # list
print(MinBenefit_ijk_sn_2) # list

MinBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, MinBenefit_ijk_sn_2, MinBenefit_ijk_sn_1))
MinBenefit_ijk_sn_ratio

CPLEX 12.8.0.0: optimal solution; objective 2485.6
75 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2518.8
40 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2521.2
28 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2452.6
37 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2480.4
22 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2488
23 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2382.2
15 dual simplex iterations (2 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2466.8
12 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2441.4
18 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2633.2
16 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2624.6
8 dual simplex iterat

CPLEX 12.8.0.0: optimal solution; objective 1974.75
24 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1851.9
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1796.7
22 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1879.5
11 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1856.55
19 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1904.85
19 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1869.6
24 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1931.7
27 dual simplex iterations (3 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1940.7
14 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1917
31 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 1870.8
8 dual simplex ite

CPLEX 12.8.0.0: optimal solution; objective 2491.4
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2475.4
6 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2420
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2413.8
12 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2586.6
26 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2475.4
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2441
21 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2473.2
13 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2488.8
24 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2516.6
10 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 2571
10 dual simplex iterations

CPLEX 12.8.0.0: optimal solution; objective 478.3490077
34 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 454.0570191
17 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 472.1924004
40 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 458.8232244
42 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 470.2854594
39 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 471.9639355
39 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 483.8973408
40 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 467.4218176
39 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 479.403459
41 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 468.5146775
36 dual simplex iterations (4 in phase I)
CPLEX 12.8.0.0: optim

CPLEX 12.8.0.0: optimal solution; objective 81075.2
36 dual simplex iterations (1 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 84510.6
32 dual simplex iterations (7 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 82054.8
28 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 84481.4
36 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 85508.4
37 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 87823.4
26 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 84701.4
43 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 85932.8
47 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 79879.2
47 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 80123.8
38 dual simplex iterations (0 in phase I)
CPLEX 12.8.0.0: optimal solution; objective 88176.2
30 dual 

[0.99983718684533873,
 0.99970962414475217,
 0.99982632621244028,
 0.99843498710495227,
 0.80533269957483244,
 0.99906098803638832,
 0.99997884863415387,
 0.99823509776679664,
 0.9999940201982932,
 0.84412156728301513,
 0.53562215874362018,
 0.99771927959922002]

In [17]:
MinBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, MinBenefit_ijk_sn_2, MinBenefit_ijk_sn_1))
MinBenefit_ijk_sn_ratio

[0.99983718684533873,
 0.99970962414475217,
 0.99982632621244028,
 0.99843498710495227,
 0.80533269957483244,
 0.99906098803638832,
 0.99997884863415387,
 0.99823509776679664,
 0.9999940201982932,
 0.84412156728301513,
 0.53562215874362018,
 0.99771927959922002]

## Other Function

In [22]:
# Increase the costs of beef and ham

cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

# 再解一次

ampl.solve()
print("New objective value:", totalcost.value())

# 印出其中項目

Buy = ampl.getVariable('Buy')
print("Buy['BEEF'].val = {}".format(Buy['BEEF'].value()))

# Display the dual value of each constraint

diet = ampl.getConstraint('diet')
for nutr in nutrients:
    print("diet['{}'].dual = {}".format(nutr, diet[nutr].dual()))

Increased costs of beef and ham.
CPLEX 12.8.0.0: optimal solution; objective 144.0120033
0 simplex iterations (0 in phase I)
New objective value: 144.01200332502074
Buy['BEEF'].val = 5.226932668329172
diet['A'].dual = 0.0
diet['C'].dual = 0.0
diet['B1'].dual = 0.0
diet['B2'].dual = 0.7999285120532
diet['NA'].dual = -0.007531172069825434
diet['CAL'].dual = 0.0
